<a href="https://colab.research.google.com/github/dawidkubicki/chatbot-tensorflow/blob/main/Chatbot_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to the GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Change root path

In [1]:
#root_path = '/content/gdrive/My Drive/Colab/'

import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "Colab/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'test_file.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/Colab/

An empty text file was created there. You can also run !pwd to confirm the current working directory.


## Download dataset



In [ ]:
#!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
#!unzip cornell_movie_dialogs_corpus.zip

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: cornell movie-dialogs corpus/movie_lines.txt  
  inflating: cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: cornell movie-dialogs corpus/raw_script_urls.txt  
  inflating: cornell movie-dialogs corpus/README.txt  
  inflating: __MACOSX/cornell movie-dialogs corpus/._README.txt  


## Firstly import all necessaries libraries


In [18]:
import numpy as np
import tensorflow as tf
import re
import time

## Import dataset to the project

In [19]:
lines = open('dataset/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conversations = open('dataset/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

## Create a dictionary that maps each line and it's ID

In [20]:
id2line = {} #init dictionary
for line in lines:
  _line = line.split(' +++$+++ ')
  if len(_line) == 5:
    id2line[_line[0]] = _line[4]

# JUST TO CHECK DATA IN GOOGLE COLAB
# retrieve key/value pairs
#els = list(id2line.items()) # explicitly convert to a list, in case it's Python 3.x

# get first inserted element 
#print(els[0])



## Create a list of all of the conversations

In [21]:
conversations_ids = []
for conversation in conversations[:-1]:
  _conversation = conversation.split(" +++$+++ ")[-1][1:-1].replace("'","").replace(" ","")
  conversations_ids.append(_conversation.split(","))
# print(conversations_ids[0])
# print(conversations_ids[1])

## Getting separately the questions and the answers

In [22]:
questions = []
answers = []
for conversation in conversations_ids:
  for i in range(len(conversation) - 1):
    questions.append(id2line[conversation[i]])
    answers.append(id2line[conversation[i+1]])

#print(questions[0])
#print(answers[0])


### Cleaning text function define

In [23]:
def clean_text(text):
  #put all the text to lowercase
  text = text.lower()
  #removing aposthrophies with re library
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"what's", "what is", text)
  text = re.sub(r"where's", "where is", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'d", " whould", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
  return text

### Cleaning questions and answers

In [24]:
clean_questions = []
clean_answers = []

for question in questions:
  clean_questions.append(clean_text(question))

for answer in answers:
  clean_answers.append(clean_text(answer))  

### Creating a dictionary that maps each word to it's number of occurencies


In [25]:
word2count = {}

for question in clean_questions:
  for word in question.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1

for answer in clean_answers:
  for word in answer.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1      

### Creating two dictionaries that map the questions words and the answers words to a unique integer

In [29]:
threshold = 20
questionwords2int = {}
word_number = 0 
for word, count in word2count.items():
  if count > threshold:
    questionwords2int[word] = word_number
    word_number += 1

answerwords2int = {}
word_number = 0 
for word, count in word2count.items():
  if count > threshold:
    answerwords2int[word] = word_number
    word_number += 1

print(answerwords2int.items())

dict_items([('can', 0), ('we', 1), ('make', 2), ('this', 3), ('quick', 4), ('and', 5), ('andrew', 6), ('are', 7), ('having', 8), ('an', 9), ('incredibly', 10), ('public', 11), ('break', 12), ('up', 13), ('on', 14), ('the', 15), ('again', 16), ('well', 17), ('i', 18), ('thought', 19), ('whould', 20), ('start', 21), ('with', 22), ('if', 23), ('that', 24), ('is', 25), ('okay', 26), ('you', 27), ('not', 28), ('part', 29), ('please', 30), ('asking', 31), ('me', 32), ('out', 33), ('so', 34), ('cute', 35), ('what', 36), ('your', 37), ('name', 38), ('no', 39), ("it's", 40), ('my', 41), ('fault', 42), ("didn't", 43), ('have', 44), ('a', 45), ('proper', 46), ('cameron', 47), ('thing', 48), ('am', 49), ('at', 50), ('mercy', 51), ('of', 52), ('particularly', 53), ('breed', 54), ('loser', 55), ('sister', 56), ('cannot', 57), ('date', 58), ('until', 59), ('she', 60), ('does', 61), ('why', 62), ('mystery', 63), ('used', 64), ('to', 65), ('be', 66), ('really', 67), ('popular', 68), ('when', 69), ('sta

### Adding the last tokens to these two dictionaries

In [36]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
  questionwords2int[token] = len(questionwords2int) + 1
 

8542
